# Загрузка данных

In [2]:
import pandas as pd

In [13]:
df = pd.read_csv('../data/train.csv').drop(columns=['index'])

In [14]:
df['class'] = pd.Categorical(df['class'])

In [15]:
df.head(3)

,text,class
0,Выплата на содержание родных детей,TAX
1,Оплата за Бритва Gillette Blue II Plus однораз...,NON_FOOD_GOODS
2,Паевой взнос в жилищно-строительный кооператив...,REALE_STATE


## Очистка датасета

In [26]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/misha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
only_text_regex = r'[^а-яА-ЯёЁ]+'
stop_words = set(stopwords.words('russian'))

def clean_text(text: str) -> str:
    return ' '.join(
        [word 
         for word in re.sub(only_text_regex, ' ', text).split() 
         if word.lower() not in stop_words]
    )

In [28]:
df['clean_text'] = df['text'].apply(clean_text)
df['clean_text']

0                        Выплата содержание родных детей
1          Оплата Бритва одноразовая шт счету июля Сумма
2      Паевой взнос жилищно строительный кооператив ц...
3      доставку д транспортом г Одинцово г Бийск дого...
4                     юридические услуги договору января
                             ...                        
395          Оплата Восточные сладости халва счету Сумма
396    транспортировку перевозка железной дороге г Бе...
397              Оплата дог Б г согл сч г счет фактура г
398        Оплата Масло оливковое жарки мл счету г Сумма
399    Оплата Крем универсальный Пантенол мл счету г ...
Name: clean_text, Length: 400, dtype: object

# Наивный байесовский классификатор

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score

In [30]:
vectorizer = TfidfVectorizer()

### На грязном тексте

In [31]:
X = vectorizer.fit_transform(df.text.tolist())
y = df['class']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [34]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.8375
F1-Score: 0.8051641372524225


### На чистом тексте

In [35]:
X = vectorizer.fit_transform(df.clean_text.tolist())
y = df['class']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()

In [38]:
y_pred = nb_model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1-Score: {f1_score(y_test, y_pred, average="macro")}')

Accuracy: 0.875
F1-Score: 0.8397206139141624
